<a href="https://colab.research.google.com/github/AdamLawsonUC/OpenVLAXAI/blob/main/Feature_Steering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check runtime
!nvidia-smi

Sat Feb 14 04:38:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             48W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip -q install -U transformers accelerate safetensors huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 46.3 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
from google.colab import files
files.upload()  #make sure to change the file name below based on the vector json that is selected

Saving steering_vectors_football.json to steering_vectors_football.json


{'steering_vectors_football.json': b'{\n  "modelId": "llama3.1-8b-it",\n  "layer": "15-resid-post-aa",\n  "index": "114045",\n  "vectorLabel": "",\n  "hookName": "blocks.15.hook_resid_post",\n  "vector": [\n    0.00885009765625,\n    0.01019287109375,\n    -0.0174560546875,\n    0.01171875,\n    -0.12109375,\n    -0.006439208984375,\n    0.0045166015625,\n    -0.00787353515625,\n    -0.0294189453125,\n    -0.01007080078125,\n    -0.01611328125,\n    0.0020294189453125,\n    0.0269775390625,\n    0.05615234375,\n    0.00799560546875,\n    0.0301513671875,\n    0.0157470703125,\n    0.005615234375,\n    -0.0625,\n    -0.009033203125,\n    0.029052734375,\n    -0.0115966796875,\n    -0.003936767578125,\n    -0.00194549560546875,\n    0.007476806640625,\n    0.009521484375,\n    -0.018310546875,\n    -0.0172119140625,\n    -0.037109375,\n    -0.003173828125,\n    -0.040771484375,\n    0.04931640625,\n    -0.0230712890625,\n    0.00872802734375,\n    -0.039794921875,\n    0.0030364990234375

In [11]:
import os
print(os.listdir("/content"))


['.config', 'steering_vectors_football.json', 'steering_vectors.json', 'sample_data']


In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.1-8B-Instruct"

if torch.cuda.is_available():
    major, minor = torch.cuda.get_device_capability(0)
    torch_dtype = torch.bfloat16 if major >= 8 else torch.float16
else:
    torch_dtype = torch.float32

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    device_map="auto",
)
model.eval()

print("Loaded on:", next(model.parameters()).device)
print("dtype:", next(model.parameters()).dtype)
print("hidden_size:", model.config.hidden_size)


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Loaded on: cuda:0
dtype: torch.bfloat16
hidden_size: 4096


In [13]:
import json
import torch

json_path = "/content/steering_vectors_football.json"

with open(json_path, "r") as f:
    data = json.load(f)

layer_idx = int(str(data.get("layer", "15")).split("-")[0]) if isinstance(data.get("layer"), str) else int(data.get("layer", 15))
hook_name = data.get("hookName", "")
index = data.get("index", "")
vec_list = data["vector"]

v = torch.tensor(vec_list, dtype=next(model.parameters()).dtype, device=next(model.parameters()).device)

print("JSON hookName:", hook_name)
print("JSON index:", index)
print("Vector shape:", tuple(v.shape))
print("Model hidden_size:", model.config.hidden_size)

assert v.numel() == model.config.hidden_size, (
    f"Vector dim {v.numel()} != hidden_size {model.config.hidden_size}. "
    "This vector may be for a different model size."
)


JSON hookName: blocks.15.hook_resid_post
JSON index: 114045
Vector shape: (4096,)
Model hidden_size: 4096


In [14]:
import torch

@torch.no_grad()
def generate(prompt: str, max_new_tokens: int = 128):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    )
    device = next(model.parameters()).device
    inputs = {k: t.to(device) for k, t in inputs.items()}

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        repetition_penalty=1.1,
    )
    answer_ids = out[0, inputs["input_ids"].shape[1]:]
    return tokenizer.decode(answer_ids, skip_special_tokens=True)


In [17]:
coefficient = 6.0  # start smaller (e.g., 1–4) if things get weird

def steering_hook(module, inputs, output):
    return output + coefficient * v

# Hook the block output at layer_idx
hook_handle = model.model.layers[layer_idx].register_forward_hook(steering_hook)

prompt = "How should I spend my free time?"

print("=== Baseline ===")
hook_handle.remove()
print(generate(prompt))

hook_handle = model.model.layers[layer_idx].register_forward_hook(steering_hook)

print("\n=== Steered (coef=%s, layer=%s, index=%s) ===" % (coefficient, layer_idx, index))
print(generate(prompt))

hook_handle.remove()


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== Baseline ===


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Spending your free time wisely can have a significant impact on your overall well-being, productivity, and happiness. Here are some suggestions to help you make the most of your free time:

1. **Pursue hobbies**: Engage in activities that bring you joy and fulfillment, such as painting, playing music, gardening, cooking, or writing.
2. **Exercise and physical activity**: Regular exercise can improve your physical and mental health. Try sports, yoga, dancing, or simply going for walks or runs.
3. **Learn something new**: Expand your knowledge by taking online courses, attending workshops, or reading books on topics that interest

=== Steered (coef=5.0, layer=15, index=114045) ===
There are many ways to spend your free time, and it all depends on what you want to achieve or what will make you happy. Here are some suggestions:

1. Improve your physical health:
    - Exercise: You can join a local club or follow a fitness routine at home.
    'Join a sports team': This can be a great way t